In [78]:
import gzip
import numpy as np
import matplotlib.pyplot as plt
from emnist import  list_datasets
from emnist import extract_training_samples
from emnist import extract_test_samples
import time

start = time.time()

im_size = 28
num_images = 6000
k = 26

images_training, labels_training = extract_training_samples('letters')
images_test, labels_test = extract_test_samples('letters')

def learn(mat):
    for i in range(im_size):
        for j in range(im_size):
            if mat[i,j] == 1:
                mat[i,j] += 1
            else:
                mat[i,j] += -1
    return mat

def readim(im):
    th = 128
    im_bool = (im > th) 
    mat = np.zeros((im_size, im_size))
    for i in range(im_size):
        for j in range(im_size):
            if im_bool[i,j]:
                mat[i,j] = 1
            else:
                mat[i,j] = 0
    return mat

alphabet = np.array(['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z'])

Wk = np.zeros((k,im_size,im_size))
for i in range(num_images):
    Wk[int(labels_training[i]) - 1] += learn(readim(images_training[i]))
    
# Fonction renvoyant ce que le reseau de neuronnes pense etre la lettre la plus probable
def Q(image):
    I = image
    phik = np.zeros(k)
    muk = np.zeros(k)
    Qk = np.zeros(k)

    for i in range(k):
        for j in range(im_size):
            for l in range(im_size):
                phik[i] += Wk[i][j][l]*I[j][l]
                if Wk[i][j][l] > 0:
                    muk[i] += Wk[i][j][l]
        Qk[i] = phik[i]/muk[i]
    return np.argmax(Qk)
    
end = time.time()
print(f"Temps d'execution du programme: {end - start} s")

Temps d'execution du programme: 7.311124086380005 s


In [79]:
start = time.time()

# Test verifiant si le reseau de neuronnes est performant
Qstat = 0
for i in range(int(num_images - 1)):
    if Q(images_test[i]) == int(labels_test[i] - 1):
        Qstat += 1
        
print(Qstat/num_images)

end = time.time()
print(f"Temps d'execution du programme: {end - start} s")

#    num_images    Q          t
#    4000          0.5805     212.73033499717712 s
#    5000          0.504      253.27191281318665 s
#    6000          0.45316    306.93508982658386 s


0.45316666666666666
Temps d'execution du programme: 306.93508982658386 s


In [58]:
im_bool = (images_test > 128)
im_bool.shape

(20800, 28, 28)